# 02: GRPO on GSM8K
Sweep hyperparameters for GRPO training on Qwen2.5-0.5B-GSM8K-SFT.
Uses binary reward (1.0/0.0) and parameterized SFT/new data mix.

In [ ]:
!pip install -q --no-deps git+https://github.com/tripathysagar/rlhf-gsm8k.git
!pip install -qUU trl

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 16.7 MB/s eta 0:00:00


In [ ]:
import torch
from types import SimpleNamespace
from datasets import load_dataset, concatenate_datasets
from trl import GRPOConfig, GRPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainerCallback
from peft import LoraConfig, get_peft_model
from gsm8k_utils import *

from transformers import set_seed
set_seed(1337)

In [ ]:
# ── Config ───────────────────────────────────────────────────────────────────
cfg = dict(
    # Model
    model_id       = "tripathysagar/Qwen2.5-0.5B-GSM8K-SFT",
    lora_r         = 16,
    lora_alpha     = 32,

    # GRPO
    num_generations         = 8,
    max_completion_length   = 256,
    max_steps               = 100,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    learning_rate           = 5e-5,
    beta                    = 0.04,
    warmup_steps            = 20,

    # wandb
    wandb_project  = "grpo-gsm8k",
    wandb_run_name = "grpo-qwen2.5-0.5B-lora",

    # sft data mix
    grpo_train_size = 1024,   # DO NOT MODIFY — fixed budget matching SFT split size
    sft_frac        = 0.5,   # 0.0 = all new, 0.5 = half/half, 1.0 = all SFT
)

In [ ]:
import wandb
from google.colab import userdata

wandb.login(key=userdata.get('wandb'))

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tripathysagar08 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Helpers

In [ ]:
class VibecheckCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            reward = logs.get('reward')
            completions = logs.get('completions')
            if reward is not None:
                print(f"Step {state.global_step} | reward: {reward:.3f}")
            if completions and len(completions) > 0:
                print(f"Step {state.global_step} | Sample:\n{completions[0][:200]}...")


def reward_fn(completions, **kwargs):
    """Binary reward: 1.0 for correct answer, 0.0 otherwise."""
    golds = kwargs["gold"]
    rewards = []
    for comp, gold in zip(completions, golds):
        try:
            ans, _ = extract_answer(comp)
            gold_int = int(float(gold.replace(",", "")))
            rewards.append(1.0 if ans == gold_int else 0.0)
        except Exception as e:
            print(f"[reward_fn error] gold={gold!r} comp={comp[:80]!r} err={e}")
            rewards.append(0.0)
    return rewards


## GRPOExperiment

In [ ]:
class GRPOExperiment:
    def __init__(self, cfg):
        self.cfg = SimpleNamespace(**cfg) if isinstance(cfg, dict) else cfg

    def load_ds(self):
        test_ds = load_dataset("openai/gsm8k", "main", split="test")
        self.test_ds = test_ds.train_test_split(test_size=0.5, seed=42)["test"]

        all_train = load_dataset("openai/gsm8k", "main", split="train").shuffle(seed=42)
        if self.cfg.sft_frac == -1:
            # for training on complete dataset
            self.train_ds = all_train
            return

        n_sft = int(self.cfg.grpo_train_size * self.cfg.sft_frac)
        n_new = self.cfg.grpo_train_size - n_sft

        sft_rows = all_train.select(range(self.cfg.grpo_train_size))
        new_rows = all_train.select(range(self.cfg.grpo_train_size, len(all_train)))

        sft_rows = sft_rows.shuffle(seed=42).select(range(n_sft)) if n_sft > 0 else sft_rows.select([])
        new_rows = new_rows.shuffle(seed=42).select(range(n_new)) if n_new > 0 else new_rows.select([])

        self.train_ds = concatenate_datasets([sft_rows, new_rows])

    def _fmt(self, ds):
        """Extract gold answers and format prompts."""
        ds = ds.map(extract_gold, remove_columns=["answer"]).rename_column("question", "prompt")
        return ds.map(lambda x: {"prompt": format_prompt(x["prompt"], self.tokenizer)})

    def fmt_ds(self):
        """Format train and test datasets for GRPO."""
        self.train_ds = self._fmt(self.train_ds)
        self.test_ds = self._fmt(self.test_ds)

    def setup_model(self):
        lora_config = LoraConfig(
            r=self.cfg.lora_r,
            lora_alpha=self.cfg.lora_alpha,
            target_modules="all-linear",
            task_type="CAUSAL_LM",
        )

        self.tokenizer = AutoTokenizer.from_pretrained(self.cfg.model_id)
        model = AutoModelForCausalLM.from_pretrained(
            self.cfg.model_id,
            dtype=torch.bfloat16,
            device_map="auto",
        )
        model.gradient_checkpointing_enable()
        self.model = get_peft_model(model, lora_config)
        self.model.print_trainable_parameters()

    def setup_trainer(self):
        wandb.init(project=self.cfg.wandb_project, name=self.cfg.wandb_run_name)

        grpo_config = GRPOConfig(
            output_dir="grpo_qwen_gsm8k",
            num_generations=self.cfg.num_generations,
            max_completion_length=self.cfg.max_completion_length,
            max_steps=self.cfg.max_steps,
            per_device_train_batch_size=self.cfg.per_device_train_batch_size,
            gradient_accumulation_steps=self.cfg.gradient_accumulation_steps,
            learning_rate=self.cfg.learning_rate,
            logging_steps=1,             # increase for longer runs (e.g. 10)
            report_to='wandb',
            beta=self.cfg.beta,
            lr_scheduler_type="cosine",
            warmup_steps=self.cfg.warmup_steps,
            weight_decay=0.01,
            save_strategy="no",          # change to "steps" for longer runs
            # save_steps=100,            # uncomment for longer runs
            # save_total_limit=2,        # uncomment for longer runs
            bf16=True,
            fp16=False,
        )

        self.trainer = GRPOTrainer(
            model=self.model,
            reward_funcs=reward_fn,
            args=grpo_config,
            train_dataset=self.train_ds,
            processing_class=self.tokenizer,
            #callbacks=[VibecheckCallback()],
        )

    def train(self):
        self.trainer.train()

    def eval(self):
        self.model = self.model.merge_and_unload()
        accuracy, _ = perf_check(self.model, self.tokenizer, self.test_ds)
        wandb.log({"eval/accuracy": accuracy})
        return accuracy

    def cleanup(self):
        wandb.finish()
        del self.model, self.trainer
        torch.cuda.empty_cache()

    def __call__(self):
        self.load_ds()
        self.setup_model()
        self.fmt_ds()
        self.setup_trainer()
        self.train()
        result = self.eval()
        self.cleanup()
        return result

## Run Experiment

In [ ]:
#exp = GRPOExperiment(cfg)
#exp()

In [ ]:
all_sweeps = [
    {"num_generations": 8},
    {"num_generations": 16},
    {"beta": 0.01},
    {"beta": 0.02},
    {"beta": 0.04},
    {"learning_rate": 1e-5},
    {"learning_rate": 5e-5},
    {"learning_rate": 1e-4},
    {"sft_frac": 0.0},
    {"sft_frac": 0.5},
    {"sft_frac": 1.0},
]

results = []
for s in all_sweeps:
    run_name = "sweep-" + "-".join(f"{k}={v}" for k, v in s.items())
    run_cfg = {**cfg, **s, "wandb_run_name": run_name}
    acc = GRPOExperiment(run_cfg)()
    results.append({**s, "accuracy": acc})
    print(f"{s} → accuracy: {acc:.4f}")

results


README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

...

Evaluating: 100%|██████████| 11/11 [00:47<00:00,  4.33s/it]


Final Accuracy: 223/660 = 33.79%


eval/accuracy,▁
profiling/Time taken: GRPOTrainer._calculate_rewards,▅▆▆▂▆▆▃▇▂▆▂▄▂▇▃▄▃▅▂█▃▃▂▄▅▅▁▆▆▂▅▄▆▃▄▇▂▇▂▃
profiling/Time taken: GRPOTrainer._get_per_token_logps_and_entropies,▇▂█▂█▂▂▂█▂▁▂▂▁▁▁▂▁▁▁▇▂▁▁▂▁▁▁▁▁▁▁▁▁█▁█▁▂▁
profiling/Time taken: GRPOTrainer._prepare_inputs,▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▇▅▁▁█▁▄▁▁▄▁▁▇▁▅▁▁▁█▁▁▁▁
profiling/Time taken: GRPOTrainer.compute_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
profiling/Time taken: GRPOTrainer.reward_fn,▃▅▂▄▃▅▄█▃▅▃▄▂▂▂▃▅▃▂▂▂▄▄▄▆▃▇▂▁▁▂▃▄▃▅▃▂▂▆▃
profiling/Time taken: GRPOTrainer.transformers.generate,▄▂▃▅████▃▄▇█▆▄▅▃▆▅▅▇██▅█▂▄▄▄▂██▃▇▅▂█▃▆█▁
train/clip_ratio/high_max,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/clip_ratio/high_mean,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/clip_ratio/low_mean,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
+23,...


{'sft_frac': 1.0} → accuracy: 0.3379


[{'num_generations': 8, 'accuracy': 0.32727272727272727},
 {'num_generations': 16, 'accuracy': 0.29545454545454547},
 {'beta': 0.01, 'accuracy': 0.3106060606060606},
 {'beta': 0.02, 'accuracy': 0.31363636363636366},
 {'beta': 0.04, 'accuracy': 0.30303030303030304},
 {'learning_rate': 1e-05, 'accuracy': 0.27121212121212124},
 {'learning_rate': 5e-05, 'accuracy': 0.30303030303030304},
 {'learning_rate': 0.0001, 'accuracy': 0.2924242424242424},
 {'sft_frac': 0.0, 'accuracy': 0.31666666666666665},
 {'sft_frac': 0.5, 'accuracy': 0.30303030303030304},
 {'sft_frac': 1.0, 'accuracy': 0.3378787878787879}]

In [ ]:
from collections import defaultdict

grouped = defaultdict(list)
for r in results:
    param_name = [k for k in r if k != "accuracy"][0]
    grouped[param_name].append(r)

for param, runs in grouped.items():
    best = max(runs, key=lambda x: x["accuracy"])
    print(f"Best {param}: {best}")


Best num_generations: {'num_generations': 8, 'accuracy': 0.32727272727272727}
Best beta: {'beta': 0.02, 'accuracy': 0.31363636363636366}
Best learning_rate: {'learning_rate': 5e-05, 'accuracy': 0.30303030303030304}
Best sft_frac: {'sft_frac': 1.0, 'accuracy': 0.3378787878787879}


## Results

### Best per sweep category

| Param | Best Value | Accuracy | Notes |
|---|---|---|---|
| `num_generations` | 16* | 36.4% | High variance (29.5–36.4%); 8 avg ~31.6% |
| `beta` | 0.02* | 31.4% | Near-flat (30.3–31.4%); not sensitive |
| `learning_rate` | 5e-5 | 30.3% | 1e-5 too slow (27.1%); 1e-4 too aggressive (29.2%) |
| `sft_frac` | 0.0* | 31.7% | Fresh data > SFT overlap; 1.0 = 33.8%** |

*\* More experiments needed — high variance between runs; multiple seeds required to confirm.*

*\*\* sft_frac=1.0 surprisingly strong — model may benefit from reinforcing known problems.*

### Best combined config (heuristic)

```python
best_cfg = {**cfg,
    "num_generations": 16,
    "beta": 0.02,
    "learning_rate": 5e-5,
    "sft_frac": 0.0,
    "max_steps": 500,  # longer final run
}
```

## Conclusion

Sweep over 100 steps, binary reward (1.0/0.0), seed=1337.

| Param | Values Tested | Best | Accuracy |
|---|---|---|---|
| `num_generations` | 8, 16 | **16** | 36.4%* |
| `beta` | 0.01, 0.02, 0.04 | **0.02** | 31.4% |
| `learning_rate` | 1e-5, 5e-5, 1e-4 | **5e-5** | 30.3% |
| `sft_frac` | 0.0, 0.5, 1.0 | **1.0** | 33.8% |

*\* High run-to-run variance observed — multiple seeds needed to confirm. These are heuristic picks from single-run sweeps.*